# data_screening Code
#### Author: Rachel Veenstra
#### Date Created: 04-03-2019

In [79]:
# Importing necessary modules

import os
import math
import pandas as pd
import numpy as np
import glob
from itertools import cycle
import matplotlib.pyplot as plt
from scipy import stats
import shutil



# Identifying directory of lab sheets for quality screening

path = str(glob.os.getcwd())

user = path.split('\\')[2]

folder = "Lab_Sheets/"

datasheets = '/Users/' + user + '/Desktop/Coding/StandardLab/' + folder

glob.os.chdir(datasheets)



# Setting loop for iterating through files WITHIN folders IN the specified folder

for x, folder, file in os.walk(datasheets):
    
    
    # Using "try" loop in the event that a folder is empty
    
    try: 
        
        
        # Looping through every file in a folder
        
        for loc in folder:
            
            
            # Identifying working folder
            
            print(loc)

            folder_loc = datasheets + loc

            glob.os.chdir(folder_loc)
            
            
            # Grabbing all csv files in specified folder

            all_datasheets = glob.glob(folder_loc + "/*.csv")
            
            
            # Creating empty lists to append with ID's that fail quality screening

            N_Re_Run = []
            U_Re_Run = []
            
            
            # Screening each sheet individually

            for sheet in all_datasheets:

                
                # Reading working file as dataframe in pandas
                
                lab_data = pd.read_csv(sheet)
                
                
                # Pulling parts of name to identify sheet for naming/moving

                sheet_name = sheet.split('\\')[-1]
                title = sheet_name.split('_')[2]
            
            
                # Checking for data - skipping remainder of code (breaking loop) if there is no data

                if np.isnan(lab_data.Absorbance[0]):
                    break

                
                # Continuing if data is found

                else:
                    
                    # Identifying sheet
                    
                    print(sheet)


                    # Creating empty lists to append with curve information
                    
                    curve_absorbance = []
                    curve_concentration = []

                    
                    # Identifying standard absorbances based on 'Type' column
                    
                    for n in range(len(lab_data)):
                        if lab_data.Type[n] == 'C':
                            
                            
                            # Appending calibration data to appropriate list for curve analysis
                            
                            curve_absorbance.append(float(lab_data.Absorbance[n]))
                            curve_concentration.append(float(lab_data.Sample_ID[n]))


                # Calculating r^2 value for two calibration data lists with lin.regress module

                slope, intercept, r_value, p_value, std_err = stats.linregress(curve_absorbance, curve_concentration)

                
                # Creating conditional for screening based on r^2 value
                
                if r_value**2 < 0.8:
                    

                    # Marking bad curve sheet and moving to 'Completed' folder to avoid re-running in the future
                    
                    shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Lab_Sheets/' + loc + '/Completed/' + 'x_BAD_CURVE_' +str(sheet_name))

                    
                    # Adding Sample ID information from bad calibration to appropriate list of re-runs

                    for i, row in lab_data.iterrows():
                        if row['Type'] == 'O':
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])
                        elif row['Type'] == 'D':
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])


                # If the calibration data is acceptable:                 
                                
                else: 
                    
                    # Defining which rows contain sample data

                    sample_data = lab_data[lab_data.Type != 'C']
                    sample_data = sample_data[sample_data.Type != 'B']
                    
                    
                    # Creating new dataframe with only ID & absorbance
                    
                    sample_data_ab = sample_data[['Sample_ID', 'Absorbance']]


                    # Grouping sample data based on unique ID

                    sample_data_g = sample_data_ab.groupby('Sample_ID')


                    # Merging and creating a new dataframe for original/duplicate variance values (max-min/max)

                    sample_data_var = sample_data_ab.merge((abs((sample_data_g.max() - sample_data_g.min())) / (sample_data_g.max())), on='Sample_ID')


                    # Create conditional based on variance column

                    for i, row in sample_data_var.iterrows():
                        if float(row['Absorbance_y']) > 0.1:

                            
                            # Adding sample ID information to appropriate list if variance is unacceptable
                            
                            if title == "URE":
                                U_Re_Run.append(row['Sample_ID'])
                            elif title == "NIT":
                                N_Re_Run.append(row['Sample_ID'])
                                

                    # Moving working sheet to 'Completed' folder in folder to avoid re-running in the future            
                                
                    shutil.move(sheet, '/Users/' + user + '/Desktop/Coding/StandardLab/Lab_Sheets/' + loc + '/Completed/' + str(sheet_name))

            
            # Pulling additional parts of name to identify sheet for naming/moving
            
            set_ = sheet_name.split('_')[1]
            title1 = sheet_name.split('_')[3]
            title2 = sheet_name.split('_')[4]
            titles = (title1, title2)
            file_name = ('_').join(titles)
            file_name  
            

            # Creating a conditional for creating new sheets with URE re-runs (if list is not empty)

            if U_Re_Run != []:
                
                
                # Prompting user to identify desired set size for re-runs

                samp_set = int(input('You have ' + str(len(U_Re_Run)/2) + ' unique samples (not including duplicates) to re-run for Ureides in ' + str(loc) + '. How many of these do you wish to run per set?'))
                print("Creating " + str(math.ceil(len(U_Re_Run)/(samp_set*2))) + " sheets for " + str(len(U_Re_Run)) + " total samples.")

                
                # Creating dataframe with re-run list
                
                U_Re_Run = pd.DataFrame(U_Re_Run)

                
                # Setting a counter to control iterations in while loop
                
                counter = 0

                
                # Creating sheets based on provided set size

                while counter <= math.ceil(len(U_Re_Run)/(samp_set*2)):

                    for i in range(math.ceil(len(U_Re_Run)/(samp_set*2))):

                        template2 = U_Re_Run[(i*samp_set*2):((1+i)*samp_set*2)]


                        # Inserting rows for blanks and curve data in ureides through a list

                        utop = []

                        utop.insert(0, 4396.660764)
                        utop.insert(0, 4396.660764)
                        utop.insert(0, 4396.660764)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 1099.165191)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 549.5825955)
                        utop.insert(0, 0.00)
                        utop.insert(0, 0.00)
                        utop.insert(0, 0.00)
                        utop.insert(0, '2_2')
                        utop.insert(0, '2_1')
                        utop.insert(0, '1_2')
                        utop.insert(0, '1_1')

                        
                        # Combining utop list and ID information into one dataframe
                        
                        template2 = pd.concat([pd.DataFrame(utop), template2], ignore_index = True)


                        # Creating values for sample type

                        seq = cycle(['O', 'D'])

                        type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']


                        
                        # Creating new columns in dataframe and appending corresponding data 
                        # ... or leaving rows empty for lab to fill in later
                        

                        template2['Type'] = [next(seq) for i in range(len(template2))]
                        
                        template2.Type[0:16] = type_list
                        
                        template2['Sample_Wt(g)'] = ''
                        
                        template2['Absorbance'] = ''
                        
                        template2 = template2[['Type', 'Sample_Wt(g)', 0, 'Absorbance']]
                        
                        template2.columns = ['Type', 'Sample_Wt(g)', 'Sample_ID', 'Absorbance']
                        
                        
                        
                        # Setting directory as original folder

                        glob.os.chdir(datasheets + '/' + str(loc))
                        

                        # Saving each ureide datasheet with a unique name based on set# 
                        # ... date is blank for user to change later when sampes are run
                        # First letter in name is changed based on number of repetitions
                        

                        if sheet_name.split('_')[0] == 'A':
                            
                            template2.to_csv('B_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            
                            
                            # User is notified of where to find sheets and what they are named
                            
                            print('New lab sheets saved as B_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv')
                         
                        
                        
                        elif sheet_name.split('_')[0] == 'B':
                            
                            template2.to_csv('C_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as C_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv')
                            
                            
                        elif sheet_name.split('_')[0] == 'C':
                            
                            template2.to_csv('D_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as D_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv')
                           
                        
                        elif sheet_name.split('_')[0] == 'D':
                            
                            template2.to_csv('E_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as E_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv')
                            
                            
                        else:
                            
                            template2.to_csv('xx_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as xx_' + str(i+1) + '_URE_' + file_name + '_00_00_00.csv')
                            
                            
                        
                        # Appending counter to reflect loop iterations
                        
                        counter = counter + 2

                        
                        
                        
            # Creating a conditional for creating new sheets with NIT re-runs (if list is not empty)
            
            if N_Re_Run != []:
                
                
                # Prompting user to identify desired set size for re-runs
                
                samp_set = int(input('You have ' + str(len(N_Re_Run)/2) + ' unique samples (not including duplicates) to re-run for Nitrates in ' + str(loc) + '. How many of these do you wish to run per set?'))
                print("Creating " + str(math.ceil(len(N_Re_Run)/(samp_set*2))) + " sheets for " + str(len(N_Re_Run)) + " total samples.")

                
                # Creating dataframe with re-run list
                
                N_Re_Run = pd.DataFrame(N_Re_Run)
                
                
                # Setting a counter to control iterations in while loop

                counter = 0
                

                # Creating sheets based on provided set size

                while counter <= math.ceil(len(N_Re_Run)/(samp_set*2)):

                    for i in range(math.ceil(len(N_Re_Run)/(samp_set*2))):

                        template2 = N_Re_Run[(i*samp_set*2):((1+i)*samp_set*2)]


                        # Inserting rows for blanks and curve data in nitrates through a list

                        ntop = []

                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 4500.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 3000.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 1500.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, 0.00)
                        ntop.insert(0, '2_2')
                        ntop.insert(0, '2_1')
                        ntop.insert(0, '1_2')
                        ntop.insert(0, '1_1')

                        
                        # Combining ntop list and ID information into one dataframe
                        
                        template2 = pd.concat([pd.DataFrame(ntop), template2], ignore_index = True)


                        # Creating values for sample type

                        seq = cycle(['O', 'D'])

                        type_list = ['B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']


                        
                        # Creating new columns in dataframe and appending corresponding data 
                        # ... or leaving rows empty for lab to fill in later

                        
                        template2['Type'] = [next(seq) for i in range(len(template2))]
                        
                        template2.Type[0:16] = type_list
                        
                        template2['Sample_Wt(g)'] = ''
                        
                        template2['Absorbance'] = ''
                        
                        template2 = template2[['Type', 'Sample_Wt(g)', 0, 'Absorbance']]
                        
                        template2.columns = ['Type', 'Sample_Wt(g)', 'Sample_ID', 'Absorbance']
                        
                        
                        
                        
                        # Setting directory as original folder

                        glob.os.chdir(datasheets + '/' + str(loc))
                        
                        

                        # Saving each ureide datasheet with a unique name based on set# 
                        # ... date is blank for user to change later when sampes are run
                        # First letter in name is changed based on number of repetitions

                        if sheet_name.split('_')[0] == 'A':
                            
                            template2.to_csv('B_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            
                            
                            # User is notified of where to find sheets and what they are named
                            
                            print('New lab sheets saved as B_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv')
                            
                            
                        elif sheet_name.split('_')[0] == 'B':
                            
                            template2.to_csv('C_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as C_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv')
                            
                            
                        elif sheet_name.split('_')[0] == 'C':
                            
                            template2.to_csv('D_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as D_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv')
                            
                            
                        elif sheet_name.split('_')[0] == 'D':
                            
                            template2.to_csv('E_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as E_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv')
                            
                            
                        else:
                            
                            template2.to_csv('xx_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv', index=False)
                            
                            print('New lab sheets saved as xx_' + str(i+1) + '_NIT_' + file_name + '_00_00_00.csv')
                            

                            
                        # Appending counter to reflect loop iterations    
                            
                        counter = counter + 2

                        
                        
    
    # Continuing to loop through folders if folder is empty and the attempted operations cannot be completed
    
    except FileNotFoundError:
        continue

MR_USB
RV_CTS
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/RV_CTS\A_1_NIT_RV_CTS_00_00_00 (bad curve).csv


You have 300.0 unique samples (not including duplicates) to re-run for Nitrates in RV_CTS. How many of these do you wish to run per set? 200


Creating 2 sheets for 600 total samples.
New lab sheets saved as B_1_NIT_RV_CTS_00_00_00.csv
New lab sheets saved as B_2_NIT_RV_CTS_00_00_00.csv
USB_Indiana
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Indiana\A_5_URE_LMR_IN_USB_03_21_19.csv


You have 28.0 unique samples (not including duplicates) to re-run for Ureides in USB_Indiana. How many of these do you wish to run per set? 14


Creating 2 sheets for 56 total samples.
New lab sheets saved as B_1_URE_LMR_IN_00_00_00.csv
New lab sheets saved as B_2_URE_LMR_IN_00_00_00.csv
USB_Iowa
USB_Minnesota
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Minnesota\A_2_NIT_LMR_MN_USB_03_15_19.csv
/Users/rveenstra/Desktop/Coding/StandardLab/Lab_Sheets/USB_Minnesota\A_2_URE_LMR_MN_USB_03_15_19.csv


You have 28.0 unique samples (not including duplicates) to re-run for Ureides in USB_Minnesota. How many of these do you wish to run per set? 13


Creating 3 sheets for 56 total samples.
New lab sheets saved as E_1_URE_LMR_MN_00_00_00.csv
New lab sheets saved as E_2_URE_LMR_MN_00_00_00.csv
New lab sheets saved as E_3_URE_LMR_MN_00_00_00.csv


You have 28.0 unique samples (not including duplicates) to re-run for Nitrates in USB_Minnesota. How many of these do you wish to run per set? 28


Creating 1 sheets for 56 total samples.
New lab sheets saved as E_1_NIT_LMR_MN_00_00_00.csv
USB_South_Dakota
Completed
Completed
Completed
Completed
Completed
Completed
